In [ ]:
import requests
from pymongo import MongoClient
import psycopg2
import pandas as pd
import re

import warnings
warnings.filterwarnings('ignore')

In [ ]:
# setup the book database using mongo

mongo = MongoClient(port=27017)

# drop the database if it already exists
mongo.drop_database('books_db')

db = mongo.books_db

# add reference to the books collection
books = db.books

In [ ]:
def getDataForPageAsJSON(pageNum):
    # this api call will retrieve mystery novels, in book format, from the Olathe Downtown library, based on the response page number
    
    query = {
        "query": "branch:\"Olathe - Downtown\" category1:\"BkAdultCirc Mystery\" contentclass:\"FICTION\" formatcode:(BK )",
        "searchType": "bl",
        "custom_edit": "false",
        "suppress": "true",
        "page": f"{pageNum}",
        "view": "grouped"
    }

    results = requests.post('https://gateway.bibliocommons.com/v2/libraries/jocolibrary/bibs/search?locale=en-US', json=query)

    return results.json()

In [ ]:
def addBooksToDB(bookList):
    # insert results in to mongo
    books.insert_many(bookList)

In [ ]:
def getBookList(resultJSON):
    books = []
    # the key for the book in the JSON is the 'id' value.  So need to select the id into a variable and select the underlying data from it.
    for entry in resultJSON['entities']['bibs']:
        id = entry;
        books.append(resultJSON['entities']['bibs'][id])

    return books

In [ ]:
def getDBCursor(dbName):
    # return the db cursor.  DB name is a parameter, since we need to use postgres to create the library db, then need library to create the tables and insert data.
    conn = psycopg2.connect(
        database=f"{dbName}", user='postgres', password='postgres', host='127.0.0.1', port='5432'
    )

    conn.autocommit = True

    cursor = conn.cursor()
    
    return cursor

In [ ]:
def createLibraryDB():
    # create the library db in postgres
    
    db_cursor = getDBCursor('postgres')

    db_cursor.execute("DROP DATABASE IF EXISTS library WITH (FORCE)")
    db_cursor.execute("CREATE DATABASE library")

    db_cursor.connection.close()

In [ ]:
def createLibraryTables():
    # create the library tables
    
    db_cursor = getDBCursor('library')

    db_cursor.execute(open("create_tables.sql", "r").read())

    db_cursor.connection.close()

In [ ]:
def loadAvailability(avail_list):
    # insert the availability reference table data
    
    db_cursor = getDBCursor('library')

    for item in avail_list:
        db_cursor.execute(f"INSERT INTO availability (availability_status) VALUES ('{item}')")

    db_cursor.connection.commit()

    # return the table data as a pandas DataFrame for in-memory lookup

    avail_df = pd.read_sql("SELECT * FROM availability", db_cursor.connection)

    db_cursor.connection.close()

    return avail_df

In [ ]:
def loadAuthors(author_list):
    # insert the author reference table data.
    # Using regex to split the name into first and last name.
    # NOTE:  Could have used 'split', but wanted to use regex for practice.

    db_cursor = getDBCursor('library')

    first_name_pattern = '^[^,]*'
    last_name_pattern = '(?<=,\s).*$'

    author_data = [tuple((name, re.findall(first_name_pattern, name)[0], re.findall(last_name_pattern, name)[0])) if ',' in name else tuple((name, None, None)) for name in author_list]

    db_args = ','.join(db_cursor.mogrify("(%s,%s,%s)", author).decode('utf-8') for author in author_data)

    db_cursor.execute("INSERT INTO author (full_name, last_name, first_name) VALUES " + (db_args))

    db_cursor.connection.commit()

    # return the table data as a pandas DataFrame for in-memory lookup
    
    author_df = pd.read_sql("SELECT * FROM author", db_cursor.connection)

    db_cursor.connection.close()

    return author_df

In [ ]:
def getAvailabilityID(availability_data, value):
    # return the availability.availability_id based on the passed status value
    return availability_data.loc[availability_data['availability_status'] == value]['availability_id'].item()

In [ ]:
def loadBooks(book_list, availability_data):
    # insert the books table data

    db_cursor = getDBCursor('library')

    book_data = [tuple((book['id'], 
                    book['briefInfo']['title'], 
                    book['briefInfo']['description'],
                    getAvailabilityID(availability_data, book['availability']['status']))) for book in book_list]
    
    db_args = ','.join(db_cursor.mogrify("(%s,%s,%s,%s)", book).decode('utf-8') for book in book_data)

    db_cursor.execute("INSERT INTO book (internal_id, title, description, availability_id) VALUES " + (db_args))

    db_cursor.connection.commit()

    # return the table data as a pandas DataFrame for in-memory lookup
    
    book_df = pd.read_sql("SELECT * FROM book", db_cursor.connection)

    db_cursor.connection.close()

    return book_df


In [ ]:
def loadBookAuthors(author_data, book_data, unique_books):
    # inser the book/author cross-reference table data
    
    db_cursor = getDBCursor('library')

    book_authors = []

    for book in unique_books:
        
        book_id = book_data.loc[book_data['internal_id'] == book['id']]['book_id'].item()
        for author in book['briefInfo']['authors']:
            author_id = author_data.loc[author_data['full_name'] == author]['author_id'].item()
            book_authors.append(tuple((book_id, author_id)))

    db_args = ','.join(db_cursor.mogrify("(%s,%s)", book_author).decode('utf-8') for book_author in book_authors)

    db_cursor.execute("INSERT INTO book_author_xref (book_id, author_id) VALUES " + (db_args))

    db_cursor.connection.commit()

    db_cursor.connection.close()

In [ ]:
print ('Starting library data load...')

# get the first page worth of data so we know how many total pages there are
firstPage = getDataForPageAsJSON(1)

totalPages = firstPage['catalogSearch']['pagination']['pages']

booksList = getBookList(firstPage)

addBooksToDB(booksList)

for pageNum in range(2, totalPages + 1):
    print(f'Getting data for page {pageNum}...')
    pageData = getDataForPageAsJSON(pageNum)
    booksList = getBookList(pageData)
    addBooksToDB(booksList)

createLibraryDB()
createLibraryTables()

availability_data = loadAvailability(books.distinct('availability.status'))
author_data = loadAuthors(books.distinct('briefInfo.authors'))

fields = {'_id':0, 'id': 1, 'briefInfo.title': 1, 'briefInfo.description': 1, 'briefInfo.authors': 1, 'availability.status': 1}
book_list = list(books.find({}, fields))

# cleaning - found that duplicate book records are returned in the api.  This list comprehension will keep unique book objects
unique_books = []
[unique_books.append(book) for book in book_list if book not in unique_books]

book_data = loadBooks(unique_books, availability_data)

loadBookAuthors(author_data, book_data, unique_books)

print('Library data load complete.')